# __Preparation of train / test split and training__

## __Splitting time series into train / test data__

In [1]:
import pandas as pd

In [1]:

#sketech on whiteboard


In [2]:
df = pd.DataFrame()

In [3]:
#train test split can be done with sklearn.model_selection.train_test_split with no leakiness

#train_test_split done in few simple lines; here, 80 % of dataset is assigned to become training data:
k = int(df.shape[0] * 0.8)

data_train = df.iloc[:k,:]
data_test = df.iloc[k:,:]

## __Exercise 1__

Write a function which takes our dataframe df as an input and returns our dataset split into X_train, y_train, X_test, y_test considering for the avoidance of leakage.

In [14]:
def train_test_ts(df, relative_train, maximal_lag, horizon):
    
    #your code here
    
    return X_train, y_train, X_test, y_test

In [4]:
def train_test_ts(df, relative_train, maximal_lag, horizon):
    '''
    Time series (ts) split function creates a train/test set under consideration of potential overlap between the two due to lag processing
    X_train, y_train, X_test, y_test = ...
    df=must contain target column as "target"; all other columns must be used as features
    percentage_train=how much of the total dataset shall be used for training; must be added between 0 - 1
    maximal_lag=out of all lag feature engineering, enter the maximal lag number
    '''
    k = int(df.shape[0] * relative_train)
    data_train = df.iloc[:k,:]
    #to avoid overlapping of train and test data, a gap of the maximal lag - 1 must be included between the two sets
    data_test = df.iloc[k+maximal_lag:,:]
    
    assert data_train.index.max() < data_test.index.min()
    
    #returns in the sequence X_train, y_train, X_test, y_test
    return (data_train.drop(columns=[f"horizon{horizon}","t CO2-e / MWh"], axis=1), data_train[f"horizon{horizon}"],
            data_test.drop(columns=[f"horizon{horizon}","t CO2-e / MWh"], axis=1), data_test[f"horizon{horizon}"])

In [5]:
file_path = '../data/train_time_features.pkl'
df = pd.read_pickle(file_path)

In [6]:
df.head()

,t CO2-e / MWh,year,minute_sin,minute_cos,hour_sin,hour_cos,weekday_sin,month_sin,month_cos,lag1,...,lag4,lag5,lag6,lag7,lag8,lag9,lag10,lag11,lag12,horizon0
2009-07-01 04:00:00,0.991217,2009,0.000000,1.000000e+00,0.866025,0.5,0.0,-0.5,-0.866025,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-07-01 04:05:00,0.000000,2009,0.500000,8.660254e-01,0.866025,0.5,0.0,-0.5,-0.866025,0.991217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-07-01 04:10:00,0.000000,2009,0.866025,5.000000e-01,0.866025,0.5,0.0,-0.5,-0.866025,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-07-01 04:15:00,0.991217,2009,1.000000,2.832769e-16,0.866025,0.5,0.0,-0.5,-0.866025,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-07-01 04:20:00,1.025701,2009,0.866025,-5.000000e-01,0.866025,0.5,0.0,-0.5,-0.866025,0.991217,...,0.991217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
X_train, y_train, X_test, y_test = train_test_ts(df, 0.8, 12, 0)

In [9]:
y_train

2009-07-01 04:00:00         NaN
2009-07-01 04:05:00         NaN
2009-07-01 04:10:00         NaN
2009-07-01 04:15:00         NaN
2009-07-01 04:20:00         NaN
                         ...   
2015-10-30 19:35:00    0.694855
2015-10-30 19:40:00    0.473708
2015-10-30 19:45:00    0.729264
2015-10-30 19:50:00    0.694855
2015-10-30 19:55:00    0.738329
Freq: 5T, Name: horizon0, Length: 666048, dtype: float64

## __Error metrics for time series data__

### __Mean Absolute Error (MAE)__

$MAE = \frac{\sum\limits_{t=1}^n | F_t - A_t |}{n} $

$F_t$: forecast value <br/>
$A_t$: actual value <br/>
$n$: sample size

### __Mean Absolute Percentage Error (MAPE)__

$MAPE = \frac{100 \%}{n}\sum\limits_{t=1}^n \frac{A_t - F_t}{A_t}$

$F_t$: forecast value <br/>
$A_t$: actual value <br/>
$n$: sample size

### __Symmetric Mean Absolute Percentage Error (SMAPE)__

$SMAPE = \frac{\sum\limits_{t=1}^n | F_t - A_t |}{\sum\limits_{t=1}^n (A_t + F_t)}$

$F_t$: forecast value <br/>
$A_t$: actual value <br/>
$n$: sample size

### __Mean Absolute Scaled Error (MASE)__

$MASE = \frac{\frac{1}{J}\sum\limits_{j} | \epsilon_j | }{\frac{1}{T-1} \sum\limits_{t=2}^T | Y_t - Y_{t-1} | }$
<br/>
<br/>
$e_j$: forecast error for given period <br/>
$J$: number of forecasts <br/>

### __Exercise__

Write a function which takes our train data (X_train, y_train), our test data (X_test, y_test), and a trained model as input and which returns the MAE as well as the MAPE of the train and test data.

In [12]:
def errors(model, X_train, y_train, X_test, y_test):
    
    #your code here
    
    print(f'train_MAE: {train_mae}'')
    print(f'test_MAE: {test_mae}')
    print(f'train_SMAPE: {train_SMAPE}'')
    print(f'test_SMAPE: {test_SMAPE}'')

In [1]:
def errors(model, X_train, y_train, X_test, y_test):

    train_mae = (sum(abs(y_train - model.predict(X_train)))/len(y_train))
    train_mape = (sum(abs((y_train - model.predict(X_train))/y_train)))*(100/len(y_train))
    train_smape = sum(abs(y_train - model.predict(X_train)))/sum(y_train + model.predict(X_train))

    test_mae = (sum(abs(y_test - model.predict(X_test)))/len(y_test))
    test_mape = (sum(abs((y_test - model.predict(X_test))/y_test)))*(100/len(y_test))
    test_smape = sum(abs(y_test - model.predict(X_test)))/sum(y_test + model.predict(X_test))

    print(f'train_MAE: {train_mae}')
    print(f'test_MAE: {test_mae}')
    print(f'train_SMAPE: {train_smape}')
    print(f'test_SMAPE: {test_smape}')

In [42]:

#grid search: beware of time series cross validation
